In [1]:
from pathlib import Path
import pandas as pd
import os

# synthesis

In [2]:
src_fldr = Path('generated_bussio')
tgt_fldr = Path('synthesis_mybboxes')
tgt_fldr.mkdir(exist_ok=False, parents=True)

In [3]:
def convert_path(pt):
    synth_old = 'images/pix2pix_result_segm_maps/firstexp/train_latest/images/'
    synth_new = 'images2/synthesis/mybboxes/train_60/cut/'
    if pt.startswith(synth_old):
        pt = pt.replace(synth_old, synth_new)
        pt = pt[:-4] + '.png'
    return pt

In [4]:
for i in range(10):
    df = pd.read_csv(src_fldr / f'train_200_{i}.csv')
    df.images = df.images.map(convert_path)
    df.to_csv(tgt_fldr / f'train_200_{i}.csv', index=False)

# synthesis augm

In [5]:
src_fldr = Path('generated_bussio')
tgt_fldr = Path('synthesis_augm_mybboxes')
tgt_fldr.mkdir(exist_ok=False, parents=True)

In [6]:
def is_orig(pt):
    return pt.startswith('images/ISIC2018_Task1-2_Training_Input')

def convert_path(pt, j):
    synth_old = 'images/pix2pix_result_segm_maps/firstexp/train_latest/images/'
    synth_new = 'images2/synthesis_augm/mybboxes/train_60/cut/'
    
    ending_old = '_semantic_synthesized_image.jpg'
    ending_new = f'_semantic_v{j}_synthesized_image.png'
    if pt.startswith(synth_old):
        pt = pt.replace(synth_old, synth_new)
        pt = pt.replace(ending_old, ending_new)
    return pt

def get_convert_path_fn(j):
    return lambda pt: convert_path(pt, j)

In [7]:
for i in range(10):
    df = pd.read_csv(src_fldr / f'train_200_{i}.csv')

    orig = df[df.images.map(is_orig)]
    gen = df[~df.images.map(is_orig)]
    
    synths = []
    for j in range(4):
        synth = gen.copy()
        convert_fn = get_convert_path_fn(j)
        synth.images = synth.images.map(convert_fn)
        synths.append(synth)
    synth = pd.concat(synths, axis=0)
    synth = synth.sample(frac=1)
    
    df2 = pd.concat([orig, synth], axis=0)
    df2.to_csv(tgt_fldr / f'train_200_{i}.csv', index=False)